In [ ]:
text = "Provérbios de Salomão, filho de Davi, rei de Israel."
tokens = text.encode(encoding='UTF-8', errors='strict')
print(text, "\n")
print(list(tokens))
print("tokens length before merge: ", len(tokens), "\n")

: 

In [3]:
def get_stats(ids):
    pair_counts = {}
    
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i+1])
        
        if pair in pair_counts:
            pair_counts[pair] += 1
        else:
            pair_counts[pair] = 1
        
    return pair_counts

def merge(ids, pair, new_token):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i+1]) == pair:
            new_ids.append(new_token)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids
            
stats = get_stats(tokens)

FREQUENCY_THRESHOLD = 4

merge_history = []

while stats:
    most_frequent_pair = max(stats, key=stats.get)
    
    highest_frequency = stats[most_frequent_pair]
    if highest_frequency < FREQUENCY_THRESHOLD:
        break
    
    new_token = max(tokens) + 1
    merge_history.append((new_token, most_frequent_pair))
    
    tokens = merge(tokens, most_frequent_pair, new_token) 
    stats = get_stats(tokens) 
    
print(list(tokens))
print("tokens length after merge: ", len(tokens), "\n")

[339, 340, 260, 114, 321, 231, 49, 32, 339, 111, 112, 341, 115, 105, 116, 297, 298, 227, 342, 339, 340, 260, 114, 321, 231, 254, 83, 271, 322, 306, 343, 229, 254, 68, 97, 272, 233, 241, 105, 32, 254, 73, 115, 114, 97, 275, 324, 238, 69, 108, 282, 97, 344, 100, 307, 227, 273, 237, 308, 308, 32, 265, 327, 328, 346, 330, 274, 227, 292, 283, 331, 266, 246, 112, 271, 97, 118, 114, 246, 267, 100, 243, 257, 331, 310, 111, 274, 347, 293, 99, 101, 98, 266, 227, 346, 243, 311, 263, 268, 291, 266, 239, 236, 115, 278, 299, 101, 122, 233, 348, 234, 250, 227, 344, 300, 105, 249, 349, 229, 344, 195, 173, 122, 297, 293, 116, 350, 330, 274, 52, 238, 97, 344, 100, 307, 227, 100, 279, 263, 117, 100, 312, 351, 97, 231, 333, 312, 117, 313, 354, 297, 98, 111, 236, 115, 278, 229, 97, 231, 106, 340, 278, 256, 53, 238, 79, 309, 227, 229, 115, 284, 321, 297, 97, 117, 109, 257, 230, 229, 355, 115, 290, 240, 233, 230, 237, 264, 116, 264, 294, 115, 99, 240, 110, 310, 229, 335, 116, 240, 284, 32, 247, 105, 257, 314

In [4]:
def decode(tokens, merge_history):
    for new_token, pair in reversed(merge_history):
        decoded_tokens = []
        i = 0
        while i < len(tokens):
            if tokens[i] == new_token:
                decoded_tokens.extend(pair)
            else:
                decoded_tokens.append(tokens[i])
            i += 1
        tokens = decoded_tokens
        
    return bytes(tokens).decode('UTF-8')

decoded_text = decode(tokens, merge_history)
print("\nDecoded Text:\n", decoded_text)


Decoded Text:
 Provérbios 1 Propósito e tema 1- Provérbios de Salomão, filho de Davi, rei de Israel. 2- Eles ajudarão a adquirir a sabedoria e a instrução; a compreender as palavras que dão entendimento; 3- a receber a instrução para proceder com sensatez, fazendo a justiça, o juízo e a retidão; 4- ajudarão a dar prudência aos ingênuos e conhecimento e bom senso aos jovens. 5- Ouça o sábio e aumente o seu saber, e quem tem discernimento obterá orientação 6- para compreender provérbios e parábolas, os ditados dos sábios e os seus enigmas. 7- O temor do Senhor é o princípio do conhecimento, mas os insensatos desprezam a sabedoria e a instrução. Prólogo: Exortações a buscar a sabedoria Advertência contra o engano 8- Ouça, meu filho, a instrução do seu pai e não abandone o ensino da sua mãe. 9- Eles serão um belo diadema sobre a sua cabeça e um colar para adornar o seu pescoço. 10- Meu filho, se pecadores tentarem seduzi‑lo, não ceda! 11- Talvez eles digam: “Venha conosco, fiquemos de toc